In [ ]:
%cd /ibex/user/slimhy/PADS/code
import os
import pickle
import numpy as np
import multiprocessing as mp
from util.misc import fps_subsample
from datasets.sampling import sample_surface_tpp
from datasets.metadata import COMPAT_TRANSFORMS, hex_to_class
from util.mesh import CUDAMesh
from util.misc import generate_colormap_colors
from tqdm import tqdm


PART_INSTANCES = "/ibex/project/c2273/3DCoMPaT/manifold_part_instances/"
OUT_DIR = "/ibex/project/c2273/PADS/3DCoMPaT/part_points"
POINTS_PER_PART = 4096
FPS_RATIO = 0.25
N_PROCS = 12
N_NODES = 4
PROC_ID = 0


def sample_part_points(pkl_file, return_stuff=False):
    mesh_dict = pickle.load(open(os.path.join(PART_INSTANCES, pkl_file), "rb"))
    model_name = pkl_file.split(".")[0]
    
    points_dict = {}
    for mesh_k, mesh in mesh_dict.items():
        cuda_mesh = CUDAMesh.from_trimesh(mesh).to("cpu")
        p_points = sample_surface_tpp(cuda_mesh, POINTS_PER_PART)
        p_points = fps_subsample(p_points / FPS_RATIO, ratio=FPS_RATIO)
        points_dict[mesh_k] = p_points.cpu().squeeze()

    # Process the point clouds
    cls_name = os.path.basename(pkl_file).split(".")[0].split("_")[0]
    stacked_points = process_point_clouds(points_dict, hex_to_class(cls_name))

    # Save points to npy file
    npy_file = f"{OUT_DIR}/{model_name}.npy"
    np.save(npy_file, stacked_points)

    if mesh_dict:
        return npy_file.replace(".pkl", ".npy"), mesh_dict
    return pkl_file

In [2]:
import trimesh


def normalize_numpy_array(points):
    """
    Normalize a numpy array of 3D points to fully fill a unit cube along all axes.
    
    :param points: numpy array of shape (N, 3) where N is the number of points
    :return: normalized numpy array of shape (N, 3)
    """
    if len(points.shape) != 2 or points.shape[1] != 3:
        raise ValueError("Input must be a numpy array of shape (N, 3)")

    # Calculate the bounding box
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)
    
    # Calculate the center of the bounding box
    center = (min_coords + max_coords) / 2
    
    # Center the points
    centered_points = points - center
    
    # Calculate the scale factors for each axis
    scale_factors = 2 / (max_coords - min_coords)
    
    # Scale the points to fill the [-1, 1] cube
    normalized_points = centered_points * scale_factors
    
    return normalized_points / 2.


def process_point_clouds(point_cloud_dict, shape_cls):
    # Concatenate all point clouds
    all_points = np.concatenate([cloud for cloud in point_cloud_dict.values()], axis=0)

    # Align the concatenated point cloud
    if shape_cls in COMPAT_TRANSFORMS:
        align_t = np.array(COMPAT_TRANSFORMS[shape_cls])
        all_points = all_points @ align_t
    
    # Normalize the concatenated point cloud
    all_points = normalize_numpy_array(all_points)
    
    # Assert that the points are within the [-0.5, 0.5] cube
    assert np.all(all_points >= -0.5) and np.all(all_points <= 0.5)
    
    # Create a K, N, 3 tensor
    n_part_points = list(point_cloud_dict.values())[0].shape[0]
    stacked_points = np.zeros((len(point_cloud_dict), n_part_points, 3))
    start_idx = 0
    for k, (_, cloud) in enumerate(point_cloud_dict.items()):
        num_points = cloud.shape[0]
        end_idx = start_idx + num_points
        
        # Store in the new stacked array
        stacked_points[k] = all_points[start_idx:end_idx]
        start_idx = end_idx
    
    return stacked_points


def visualize_pointcloud(
    stacked_points,
    point_radius=0.005,
    colormap="viridis",
    alpha=1.0,
):
    """
    Create a scene combining the main mesh, its bounding boxes, and points for each part.
    Bounding boxes and corresponding points share the same color.
    """
    scene = trimesh.Scene()

    # Generate colors for parts
    n_parts = len(stacked_points)
    colors = generate_colormap_colors(n_parts, colormap_name=colormap, alpha=alpha)

    for i, color in enumerate(colors):
        # Add points for this part using the same color
        part_points = stacked_points[i]
        
        # Plot every point as a sphere
        for point in part_points:
            sphere = trimesh.creation.uv_sphere(radius=point_radius)
            sphere.apply_translation(point)
            sphere.visual.face_colors = np.array(color) * 255
            scene.add_geometry(sphere)

    return scene

In [ ]:
pkl_files = [f for f in os.listdir(PART_INSTANCES) if f.endswith(".pkl")]
pkl_files.sort()


def chunk(l, n):
    """
    Chunk a list into n equally sized sublists.
    And NOT chunk it into sublists of size n.
    """
    return [l[i*len(l)//n:(i+1)*len(l)//n] for i in range(n)]

pkl_files = chunk(pkl_files, N_NODES)[PROC_ID]

# Create a pool of workers
with mp.Pool(processes=N_PROCS) as pool:
    # Use imap to process files and update the progress bar
    for _ in tqdm(pool.imap_unordered(sample_part_points, pkl_files), total=len(pkl_files), desc="Processing files"):
        pass